# Eksplorasi ipywidget

In [ ]:
import ipywidgets as widgets
import geopandas
import numpy
from IPython.display import clear_output
import geemap
import ee
from ipyleaflet import WidgetControl
import ipyleaflet
from traitlets import traitlets

In [ ]:
# ee.Authenticate
ee.Initialize()

In [ ]:
gdf = geopandas.read_file('./samples/Aug-06-2021_Borneo_2016_stratifiedsamples_15_1000.shp')
gdf['ID'] = numpy.arange(len(gdf))
id_list = gdf['ID'].tolist()

## `Button` <em>widgets</em> untuk memilih <em>next</em> dan <em>previous features</em>

In [ ]:
up_button = widgets.Button(description = "Up")
down_button = widgets.Button(description = "Down")
init_button = widgets.Button(description = "Initial")
out = widgets.Output()

count = 0

def init_clicked(_):
    with out:
        count = 0
        selected = gdf[gdf['ID'] == count]
        clear_output()
        print(selected)

init_button.on_click(init_clicked)

def up_button_clicked(_, incr = 1):
    with out:
        global count
        count += incr
        selected = gdf[gdf['ID'] == count]
        clear_output()
        print(selected)

up_button.on_click(up_button_clicked)

def down_button_clicked(_):
    return up_button_clicked(_, -1)

down_button.on_click(down_button_clicked)

buttons = widgets.HBox([init_button, up_button, down_button])
widgets.VBox([buttons, out])

## Memilih id dari list dengan `Button`

In [ ]:
# print id from list

up_button = widgets.Button(description = "Up")
down_button = widgets.Button(description = "Down")
initial_button = widgets.Button(description = "Initial")
out = widgets.Output()

count = 0

def init_clicked(_):
    with out:
        count = 0
        selected = selected = id_list[count]
        clear_output()
        print(selected)

init_button.on_click(init_clicked)

def up_button_clicked(_, incr = 1):
    with out:
        global count
        count += incr
        selected = id_list[count]
        clear_output()
        print(selected)

up_button.on_click(up_button_clicked)

def down_button_clicked(_):
    return up_button_clicked(_, -1)

down_button.on_click(down_button_clicked)

buttons = widgets.HBox([init_button, up_button, down_button])
widgets.VBox([buttons, out])

## `Button` <em>widgets</em> untuk query objek pada peta ##

In [ ]:
widget_width = "250px"
padding = "0px 0px 0px 4px"  # upper, right, bottom, left

In [ ]:
initButton = widgets.Button(
    description = 'Initial feature',
    button_style = 'info',
    tooltip = 'Initial feature',
    icon = 'toggle-on'
)

showButton = widgets.Button(
    description = 'Show map!',
    button_style = 'info',
    tooltip = 'Show map!',
    icon = 'toggle-on'
)

nextButton = widgets.Button(
    description = 'Next feature',
    button_style = 'info',
    tooltip = 'Next feature',
    icon = 'toggle-right'
)

prevButton = widgets.Button(
    description = 'Prev. feature',
    button_style = 'info',
    tooltip = 'Prev. feature',
    icon = 'toggle-left'
)

toolbar_widget = widgets.VBox()
toolbar_widget.children = [
    widgets.HBox([initButton, prevButton, nextButton])
]

toolbar_ctrl = WidgetControl(widget = toolbar_widget, padding = padding, widget_width = widget_width, position = "topright")
display(toolbar_widget)

In [ ]:
map = geemap.Map()
map.add_control(toolbar_ctrl)



## `widgets.interact` (it works, but...) ##
Note:

1. Menggunakan `widgets.BoundedIntText` untuk memilih object berdasarkan ID dan menggunakan `widgets.interact`.
2. Menggunakan `widgets.interact` untuk memilih kelas penutup lahan dan menggunakan `widgets.interact_manual` untuk melakukan assignment berdasarkan tombol trigger.

Isu:
1. Metode ini efektif ketika digunakan untuk mengisi atribut **satu persatu**, secara **berurutan** dan sekali jalan. Jika terdapat objek titik yang terlewati, maka akan cukup repot untuk melakukan pencarian dikarenakan query data menggunakan interval object ID = 1 (bukan next feature).
2. `ipyleaflet.WidgetControl` muncul di dua tempat.

In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
# geemap

# widgets
oid_selector = widgets.BoundedIntText(0,0,1000,1)
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)

# interact
def filt_row(oid):
    map = geemap.Map()
    gdf_obj = gdf.loc[gdf['ID'] == oid]
    ee_obj = geemap.geopandas_to_ee(gdf_obj)
    map.addLayer(ee_obj)
    # map.add_control(toolbar_ctrl)
    map.centerObject(ee_obj, 10)
    print('Selected ID: {}'.format(oid))
    return map

def assign_f(classname):
    oid = oid_selector.value
    gdf.loc[gdf['ID'] == oid, 'Class'] = classname

widgets.interact(filt_row, oid = oid_selector)
widgets.interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))

In [ ]:
# ipyleaflet

# widgets
oid_selector = widgets.BoundedIntText(0,0,1000,1)
class_assign = widgets.Dropdown(options = ['Forest','No-forest'],
                                description = 'Class name:',
                                value = None)
toolbar_widget = widgets.VBox([oid_selector, class_assign])
toolbar_control = WidgetControl(widget = toolbar_widget, position = "topleft")

# interact
def filt_row(oid):
    m = ipyleaflet.Map()
    gdf_obj = gdf.loc[gdf['ID'] == oid]
    # ee_obj = geemap.geopandas_to_ee(gdf_obj)
    geo_obj = ipyleaflet.GeoData(geo_dataframe = gdf_obj)
    m.add_layer(geo_obj)
    lonCent = (gdf_obj.bounds.maxx + gdf_obj.bounds.minx).mean()/2
    latCent = (gdf_obj.bounds.maxy + gdf_obj.bounds.miny).mean()/2
    m.center = (latCent,lonCent)
    m.add_control(toolbar_control)
    # map.centerObject(ee_obj, 10)
    print('Selected ID: {}'.format(oid))
    return m

def assign_f(classname):
    oid = oid_selector.value
    gdf.loc[gdf['ID'] == oid, 'Class'] = classname

widgets.interact(filt_row, oid = oid_selector)
widgets.interact_manual(assign_f, classname = class_assign, gdf = widgets.fixed(gdf))

In [ ]:
gdf.head()